## Imports

In [1]:
from bs4 import BeautifulSoup
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import pandas as pd
import requests
import re

## Selenium Setup

In [2]:
def openChrome():
    portz = 9995
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument(f'--remote-debugging-port={portz}')
    chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"])
    chromeOptions.add_argument("--disable-blink-features=AutomationControlled")
    chromeOptions.add_argument("--disable-blink-features")
    # chromeOptions.add_argument("--headless")

    prefs = {"profile.managed_default_content_settings.images": 2}
    chromeOptions.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(executable_path=r"/Users/kylewang/Desktop/MLSA/chromedriver", options=chromeOptions)    
    driver.set_window_rect(0, 0, width=360, height=600)
    driver.set_page_load_timeout(66)
    return driver

## Scraper Code

In [3]:
# On the new tab that opens - login to Reonomy
# Fill in your search criteria and boundary
# Hit the enter key on the code below and the scrape will start

In [ ]:
driver = openChrome()

url =  f'https://app.reonomy.com/!/home'
driver.get(url)
sleep(1)
element = WebDriverWait(driver, 33).until(
    EC.presence_of_element_located((By.XPATH, f"//input[@id='1-email']"))
)
sleep(1)

number_page = int(input("Login Select And Input Number Page: "))

#scott@MLSAVentures.com  MLSAVentures123%
counter = 0
while 1:
    output_file_path = 'reonomy.xlsx'
    with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
        pass

    url = driver.current_url
    # https://app.reonomy.com/!/search/c2b7324d-db03-4d87-85f1-1e12405f3dab?topLeft=34.66805109771855&topLeft=-99.35776003125&bottomRight=30.697968087488565&bottomRight=-91.88705690625&page=1

    for iz in range(1, number_page+1):
        driver.set_window_rect(0, 0, width=360, height=600)
        url_page = url + f"?page={iz}"
        driver.get(url_page)
        sleep(2)
        element = WebDriverWait(driver, 66).until(
            EC.presence_of_element_located((By.XPATH, f"//main[@id='summary-cards-list-container']"))
        )

        page_source  = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        data = soup.find_all("div",attrs={"data-testid": "summary-card"})
        for item in data:
            counter += 1
            link = "https://app.reonomy.com" + item.find("a",attrs={"target": "_blank"}).get("href") + "/ownership"
#             print(link)
            adress = item.find("h6").text

            pattern = re.compile(r'MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-6$')
            datas = item.find_all("div", attrs={"class": pattern})
            data_1 = datas[0]
            data_2 = datas[1]

            retail, built, sold, tax, owner = "", "", "", "", ""
            infos = data_1.find_all("p")
            for info in infos:
                info = info.text.replace("\n", "").strip()
                if "Built in" in info:
                    built = info.split("Built in ")[1]
                elif "SF" in info:
                    retail = info
                elif "tax amount" in info:
                    tax = info.split("$")[1]
                elif "Sold on" in info:
                    sold = info.split("Sold on ")[1]

#             print(adress, retail, built, sold, tax)

            owner = data_2.find_all("div")[1]
            if owner != None:
                owner = owner.text
#             print(owner)

            try:
                driver.get(link)
                sleep(2)
                element = WebDriverWait(driver, 66).until(
                    EC.presence_of_element_located((By.XPATH, f"//a[@id='property-details-tab-ownership']"))
                )
                hrrrr = True
                try:
                    element = WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[@data-testid='owner-card-id']//a[@role='button']"))
                    )
                except:
                    hrrrr = False
                contact_count = 0
                if hrrrr == True:
                    driver.execute_script("arguments[0].click();", element) 
                    element = WebDriverWait(driver, 11).until(
                        EC.presence_of_element_located((By.XPATH, f"(//span[@class='MuiButton-label'])[2]"))
                    )
                    sleep(1)
                    page_source_c  = driver.page_source
                    all_ct = BeautifulSoup(page_source_c, 'html.parser')
                    all_ct = all_ct.find_all("div",attrs={"class": "MuiGrid-root MuiGrid-container"})

                    bt_c = 0
                    for chils in all_ct:
                        name, relationship = "", ""
                        phone, email, addresses = "", "", ""
                        name = chils.find("a",attrs={"style": "color: inherit; text-decoration: none;"})
                        if name != None:
                            name = name.text
                        else:
                            continue
                        relationship = chils.find("p",attrs={"class": "MuiTypography-root MuiTypography-body2 MuiTypography-colorTextSecondary"})
                        if relationship != None:
                            relationship = relationship.text
#                         print(name, relationship)

                        try:
                            element = WebDriverWait(driver, 11).until(
                                EC.presence_of_element_located((By.XPATH, f"//input[@placeholder='Search by Name, Role/Title, or Company']"))
                            )
                            element.click()
                            element.clear()
                            element.send_keys(name)
                            sleep(2)
                            element = WebDriverWait(driver, 11).until(
                                EC.presence_of_element_located((By.XPATH, f"(//span[@class='MuiButton-label'])[2]"))
                            )
                            driver.execute_script("arguments[0].scrollIntoView();", element) 
                            sleep(1)
                            element.click()
                            element = WebDriverWait(driver, 6).until(
                                EC.presence_of_element_located((By.XPATH, f"//h6[contains(text(),'{name}')]"))
                            )
                            page_source_c  = driver.page_source
                            chils_1 = BeautifulSoup(page_source_c, 'html.parser')
                            
#                             phone = chils_1.find("p",attrs={"data-testid": "people-contact-phone-id"})
#                             if phone != None:
#                                 phone = phone.text

#                             email = chils_1.find("p",attrs={"data-testid": "people-contact-email-id"})
#                             if email != None:
#                                 email = email.text
#                             print(email)
                            
                            phone_data = chils_1.find_all("p",attrs={"data-testid": "people-contact-phone-id"})
                            for ph_ in phone_data:
                                if len(phone) != 0:
                                    phone = phone + ',' + ph_.text
                                else:
                                    phone = ph_.text
                            print(phone)
                            
                            email_data = chils_1.find_all("p",attrs={"data-testid": "people-contact-email-id"})
                            for em_ in email_data:
                                if len(email) != 0:
                                    email = email + ',' + em_.text
                                else:
                                    email = em_.text
                            print(email)
                            
                            addresses = chils_1.find("p",attrs={"data-testid": "people-contact-address-id"})
                            if addresses != None:
                                addresses = addresses.text
#                             print(phone, email, addresses)
                            sleep(1)
                            element = WebDriverWait(driver, 11).until(
                                EC.presence_of_element_located((By.XPATH, f"//button[@class='MuiButtonBase-root MuiIconButton-root']"))
                            ).click()
                            bt_c += 1

                            list_write = []
                            list_write.append([adress, retail, built, sold, tax,  owner, name, relationship, phone, email, addresses, link])
#                             print(list_write)
                            rows = list(zip(*list_write))
                            cols_names = ['Address', 'About', 'Year Built', 'Sold On', 'Tax Amount', 'Owner', 'Contact Name', 'Contact Relationship', 'Phone Number', 'Email', 'Addresses', 'Link']
                            df_new_rows = pd.DataFrame(dict(zip(cols_names, rows)))
#                             print(df_new_rows)
                            df_existing = pd.read_excel(output_file_path)
                            df_combined = pd.concat([df_existing, df_new_rows], ignore_index=True)
                            df_combined.to_excel(output_file_path, index=False)

                            if bt_c > 1:
                                break
                            sleep(1)
                            contact_count += 1
                            print("Page Number - ", iz)
                            print("Property Count - ", counter)
                            print("Contact Count for Property - ", contact_count)
                            print("------------------------------------------------")
                        except:
                            try:
                                element = WebDriverWait(driver, 11).until(
                                    EC.presence_of_element_located((By.XPATH, f"//button[@class='MuiButtonBase-root MuiIconButton-root']"))
                                ).click()
                            except:
                                pass
                else:
                    page_source_c  = driver.page_source
                    chils = BeautifulSoup(page_source_c, 'html.parser')

                    chils = chils.find_all("div",attrs={"data-testid": "owner-card-id"})
                    bt_c = 0
                    for ci in chils:
                        name, relationship = "", ""
                        phone, email, addresses = "", "", ""
                        try:
                            name = ci.find("a",attrs={"target": "_blank"})
                            if name != None:
                                name = name.text
#                             print(name, relationship)

                            addresses = ci.find("p",attrs={"data-testid": "people-contact-address-id"})
                            if addresses != None:
                                addresses = addresses.text
                            phone_data = chils_1.find_all("p",attrs={"data-testid": "people-contact-phone-id"})
                            for ph_ in phone_data:
                                if len(phone) != 0:
                                    phone = phone + ',' + ph_.text
                                else:
                                    phone = ph_.text
                            print(phone)
                            
                            email_data = chils_1.find_all("p",attrs={"data-testid": "people-contact-email-id"})
                            for em_ in email_data:
                                if len(email) != 0:
                                    email = email + ',' + em_.text
                                else:
                                    email = em_.text
                            print(email)
#                             print(phone, email, addresses)
                            bt_c += 1

                            list_write = []
                            list_write.append([adress, retail, built, sold, tax,  owner, name, relationship, phone, email, addresses, link])
#                             print(list_write)
                            rows = list(zip(*list_write))
                            cols_names = ['Address', 'About', 'Year Built', 'Sold On', 'Tax Amount', 'Owner', 'Contact Name', 'Contact Relationship', 'Phone Number', 'Email', 'Addresses', 'Link']
                            df_new_rows = pd.DataFrame(dict(zip(cols_names, rows)))
#                             print(df_new_rows)

                            df_existing = pd.read_excel(output_file_path)
                            df_combined = pd.concat([df_existing, df_new_rows], ignore_index=True)
                            df_combined.to_excel(output_file_path, index=False)
                            contact_count += 1
                            print("Page Number - ", iz)
                            print("Property Count - ", counter)
                            print("Contact Count for Property - ", contact_count)
                            print("------------------------------------------------")
                            if bt_c > 5:
                                break
                        except:
                            pass
            except:# ZeroDivisionError  as e:
                print("Fail Contract")
                driver.get(link)

print("Done with scrape")

/var/folders/2p/mkvb6kbd08l30n0fx72jcwz80000gn/T/ipykernel_97166/918096371.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"/Users/kylewang/Desktop/MLSA/chromedriver", options=chromeOptions)
